1 subject. Feature extraction. Classification for each frequency band.

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
import csv

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s32.dat"
subject_id = 32

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)
#df_videos

In [116]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [117]:
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx


In [118]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft


In [119]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [120]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

In [121]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

aux_delta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
1,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
2,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
3,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
4,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2556,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2557,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2558,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894


In [122]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

aux_theta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
1,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
2,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
3,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
4,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2556,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2557,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2558,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363


In [123]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

aux_alpha

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
1,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
2,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
3,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
4,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2556,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2557,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2558,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939


In [124]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

aux_beta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
1,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
2,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
3,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
4,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2556,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2557,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2558,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206


In [125]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

aux_gamma

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
1,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
2,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
3,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
4,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2556,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2557,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2558,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092


In [126]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

aux_labels

,valence,arousal,dominance,liking
0,8.13,4.83,9.0,4.87
1,8.13,4.83,9.0,4.87
2,8.13,4.83,9.0,4.87
3,8.13,4.83,9.0,4.87
4,8.13,4.83,9.0,4.87
...,...,...,...,...
2555,7.15,4.03,9.0,1.88
2556,7.15,4.03,9.0,1.88
2557,7.15,4.03,9.0,1.88
2558,7.15,4.03,9.0,1.88


In [127]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

In [128]:
data = data_delta.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'anger', 'pleasure', 'joy']
'''
label = np.where((data['valence'] <4.5) & (data['arousal']<4.5), 'sadness')
label = np.where((data['valence'] <4.5) & (data['arousal'] > 4.5), 'pleasure')
label = np.where((data['valence'] >4.5) & (data['arousal']<4.5), 'anger')
label = np.where((data['valence'] >4.5) & (data['arousal'] > 4.5), 'joy')
'''
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_delta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_delta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_delta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_delta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_delta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_delta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_delta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_delta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_delta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_delta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_delta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_delta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_delta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_delta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_delta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_delta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_delta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_delta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_delta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_delta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_delta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_delta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_delta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_delta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_delta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_delta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_delta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_delta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_delta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_delta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_delta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_delta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_delta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_delta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_delta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_delta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100

In [129]:
data = data_theta.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_theta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_theta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_theta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_theta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_theta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_theta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_theta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_theta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_theta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_theta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_theta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_theta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_theta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_theta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_theta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_theta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_theta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_theta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_theta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_theta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_theta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_theta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_theta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_theta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_theta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_theta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_theta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_theta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_theta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_theta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_theta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_theta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_theta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_theta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_theta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_theta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100

In [130]:
data = data_alpha.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_alpha = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_alpha = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_alpha = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_alpha = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_alpha = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_alpha = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_alpha = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_alpha = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_alpha = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_alpha = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_alpha = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_alpha = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_alpha = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_alpha = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_alpha = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_alpha = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_alpha = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_alpha = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_alpha = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_alpha = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_alpha = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_alpha = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_alpha = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_alpha = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_alpha = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_alpha = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_alpha = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_alpha = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_alpha = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_alpha = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_alpha = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_alpha = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_alpha = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_alpha = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_alpha = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_alpha = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100

In [131]:
data = data_beta.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

cparam_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_beta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_beta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_beta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_beta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_beta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_beta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_beta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_beta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_beta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_beta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_beta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_beta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_beta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_beta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_beta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_beta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_beta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_beta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_beta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_beta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_beta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_beta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_beta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_beta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_beta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_beta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_beta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_beta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_beta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_beta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_beta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_beta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_beta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_beta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_beta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_beta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])     

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100

In [132]:
data = data_gamma.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_gamma = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_gamma = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_gamma = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_gamma = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_gamma = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_gamma = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_gamma = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_gamma = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_gamma = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_gamma = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_gamma = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_gamma = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_gamma = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_gamma = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_gamma = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_gamma = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_gamma = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_gamma = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_gamma = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_gamma = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_gamma = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_gamma = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_gamma = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_gamma = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_gamma = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_gamma = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_gamma = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_gamma = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_gamma = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_gamma = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_gamma = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_gamma = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_gamma = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_gamma = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_gamma = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_gamma = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100

In [133]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Sadness precision',
             'Pleasure precision',
             'Anger precision',
             'Joy precision',
             'Sadness recall',
             'Pleasure recall',
             'Anger recall',
             'Joy recall',
             'Sadness f1-score',
             'Pleasure f1-score',
             'Anger f1-score',
             'Joy f1-score']

data_CSV = [subject_id, 5, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_sadness_delta, rf_prec_pleasure_delta, rf_prec_anger_delta, rf_prec_joy_delta, rf_rec_sadness_delta, rf_rec_pleasure_delta, rf_rec_anger_delta, rf_rec_joy_delta, rf_f1_sadness_delta, rf_f1_pleasure_delta, rf_f1_anger_delta, rf_f1_joy_delta], [subject_id, 5, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_sadness_delta, svm_prec_pleasure_delta, svm_prec_anger_delta, svm_prec_joy_delta, svm_rec_sadness_delta, svm_rec_pleasure_delta, svm_rec_anger_delta, svm_rec_joy_delta, svm_f1_sadness_delta, svm_f1_pleasure_delta, svm_f1_anger_delta, svm_f1_joy_delta], [subject_id, 5, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_sadness_delta, knn_prec_pleasure_delta, knn_prec_anger_delta, knn_prec_joy_delta, knn_rec_sadness_delta, knn_rec_pleasure_delta, knn_rec_anger_delta, knn_rec_joy_delta, knn_f1_sadness_delta, knn_f1_pleasure_delta, knn_f1_anger_delta, knn_f1_joy_delta], [subject_id, 5, "STFT", "theta", "-", "Random Forest", rf_acc_theta, rf_prec_sadness_theta, rf_prec_pleasure_theta, rf_prec_anger_theta, rf_prec_joy_theta, rf_rec_sadness_theta, rf_rec_pleasure_theta, rf_rec_anger_theta, rf_rec_joy_theta, rf_f1_sadness_theta, rf_f1_pleasure_theta, rf_f1_anger_theta, rf_f1_joy_theta], [subject_id, 5, "STFT", "theta", "-", "SVM", svm_acc_theta, svm_prec_sadness_theta, svm_prec_pleasure_theta, svm_prec_anger_theta, svm_prec_joy_theta, svm_rec_sadness_theta, svm_rec_pleasure_theta, svm_rec_anger_theta, svm_rec_joy_theta, svm_f1_sadness_theta, svm_f1_pleasure_theta, svm_f1_anger_theta, svm_f1_joy_theta], [subject_id, 5, "STFT", "theta", "-", "kNN", knn_acc_theta, knn_prec_sadness_theta, knn_prec_pleasure_theta, knn_prec_anger_theta, knn_prec_joy_theta, knn_rec_sadness_theta, knn_rec_pleasure_theta, knn_rec_anger_theta, knn_rec_joy_theta, knn_f1_sadness_theta, knn_f1_pleasure_theta, knn_f1_anger_theta, knn_f1_joy_theta], [subject_id, 5, "STFT", "alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_sadness_alpha, rf_prec_pleasure_alpha, rf_prec_anger_alpha, rf_prec_joy_alpha, rf_rec_sadness_alpha, rf_rec_pleasure_alpha, rf_rec_anger_alpha, rf_rec_joy_alpha, rf_f1_sadness_alpha, rf_f1_pleasure_alpha, rf_f1_anger_alpha, rf_f1_joy_alpha], [subject_id, 5, "STFT", "alpha", "-", "SVM", svm_acc_alpha, svm_prec_sadness_alpha, svm_prec_pleasure_alpha, svm_prec_anger_alpha, svm_prec_joy_alpha, svm_rec_sadness_alpha, svm_rec_pleasure_alpha, svm_rec_anger_alpha, svm_rec_joy_alpha, svm_f1_sadness_alpha, svm_f1_pleasure_alpha, svm_f1_anger_alpha, svm_f1_joy_alpha], [subject_id, 5, "STFT", "alpha", "-", "kNN", knn_acc_alpha, knn_prec_sadness_alpha, knn_prec_pleasure_alpha, knn_prec_anger_alpha, knn_prec_joy_alpha, knn_rec_sadness_alpha, knn_rec_pleasure_alpha, knn_rec_anger_alpha, knn_rec_joy_alpha, knn_f1_sadness_alpha, knn_f1_pleasure_alpha, knn_f1_anger_alpha, knn_f1_joy_alpha], [subject_id, 5, "STFT", "beta", "-", "Random Forest", rf_acc_beta, rf_prec_sadness_beta, rf_prec_pleasure_beta, rf_prec_anger_beta, rf_prec_joy_beta, rf_rec_sadness_beta, rf_rec_pleasure_beta, rf_rec_anger_beta, rf_rec_joy_beta, rf_f1_sadness_beta, rf_f1_pleasure_beta, rf_f1_anger_beta, rf_f1_joy_beta], [subject_id, 5, "STFT", "beta", "-", "SVM", svm_acc_beta, svm_prec_sadness_beta, svm_prec_pleasure_beta, svm_prec_anger_beta, svm_prec_joy_beta, svm_rec_sadness_beta, svm_rec_pleasure_beta, svm_rec_anger_beta, svm_rec_joy_beta, svm_f1_sadness_beta, svm_f1_pleasure_beta, svm_f1_anger_beta, svm_f1_joy_beta], [subject_id, 5, "STFT", "beta", "-", "kNN", knn_acc_beta, knn_prec_sadness_beta, knn_prec_pleasure_beta, knn_prec_anger_beta, knn_prec_joy_beta, knn_rec_sadness_beta, knn_rec_pleasure_beta, knn_rec_anger_beta, knn_rec_joy_beta, knn_f1_sadness_beta, knn_f1_pleasure_beta, knn_f1_anger_beta, knn_f1_joy_beta], [subject_id, 5, "STFT", "gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_sadness_gamma, rf_prec_pleasure_gamma, rf_prec_anger_gamma, rf_prec_joy_gamma, rf_rec_sadness_gamma, rf_rec_pleasure_gamma, rf_rec_anger_gamma, rf_rec_joy_gamma, rf_f1_sadness_gamma, rf_f1_pleasure_gamma, rf_f1_anger_gamma, rf_f1_joy_gamma], [subject_id, 5, "STFT", "gamma", "-", "SVM", svm_acc_gamma, svm_prec_sadness_gamma, svm_prec_pleasure_gamma, svm_prec_anger_gamma, svm_prec_joy_gamma, svm_rec_sadness_gamma, svm_rec_pleasure_gamma, svm_rec_anger_gamma, svm_rec_joy_gamma, svm_f1_sadness_gamma, svm_f1_pleasure_gamma, svm_f1_anger_gamma, svm_f1_joy_gamma], [subject_id, 5, "STFT", "gamma", "-", "kNN", knn_acc_gamma, knn_prec_sadness_gamma, knn_prec_pleasure_gamma, knn_prec_anger_gamma, knn_prec_joy_gamma, knn_rec_sadness_gamma, knn_rec_pleasure_gamma, knn_rec_anger_gamma, knn_rec_joy_gamma, knn_f1_sadness_gamma, knn_f1_pleasure_gamma, knn_f1_anger_gamma, knn_f1_joy_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment5_1.csv", mode="a", header=False, index=False)